"""
# Model Evaluation

This notebook evaluates model performance and generates metrics for:
- Accuracy, Precision, Recall, F1-Score
- Confusion matrices
- ROC curves
- Performance visualizations
"""

### 1. Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_curve, auc, precision_recall_curve
)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from models.anomaly_detector import HybridAnomalyDetector
from models.sentiment_engine import SentimentAnalyzer
from models.risk_scorer import BayesianRiskScorer
from config import MODELS_DIR

print("✓ Imports successful")

### 2. Load Models


In [ ]:
# Load trained anomaly detector
detector = HybridAnomalyDetector()
model_path = MODELS_DIR / 'anomaly_detector.pkl'

if model_path.exists():
    detector.load(model_path)
    print(f"✓ Loaded anomaly detector from {model_path}")
else:
    print("⚠️ Training new model (no saved model found)")
    # Generate training data
    np.random.seed(42)
    X_train = np.vstack([
        np.random.randn(900, 10),
        np.random.randn(100, 10) * 3 + 5
    ])
    detector.fit(X_train)

In [ ]:
# Initialize other models
sentiment_analyzer = SentimentAnalyzer()
risk_scorer = BayesianRiskScorer()

print("✓ All models loaded")

### 3. Generate Test Data


In [ ]:
def generate_test_data(n_samples=500):
    """Generate test data with known labels"""
    np.random.seed(123)
    
    # Normal samples
    X_normal = np.random.randn(int(n_samples * 0.85), 10)
    y_normal = np.zeros(len(X_normal))
    
    # Anomalies
    X_anomaly = np.random.randn(int(n_samples * 0.15), 10) * 3 + 5
    y_anomaly = np.ones(len(X_anomaly))
    
    # Combine and shuffle
    X_test = np.vstack([X_normal, X_anomaly])
    y_test = np.hstack([y_normal, y_anomaly])
    
    indices = np.random.permutation(len(X_test))
    return X_test[indices], y_test[indices]

X_test, y_test = generate_test_data()
print(f"✓ Generated {len(X_test)} test samples")
print(f"  Normal: {(y_test == 0).sum()}")
print(f"  Anomalies: {(y_test == 1).sum()}")

### 4. Anomaly Detection Evaluation

In [ ]:
print("\n" + "="*60)
print("ANOMALY DETECTION EVALUATION")
print("="*60)

# Predict
y_pred = detector.predict(X_test)
y_scores = detector.predict(X_test, return_scores=True)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Anomaly']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'],
            cbar_kws={'label': 'Count'})
ax.set_title('Confusion Matrix - Anomaly Detection', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')

In [ ]:
# Add accuracy annotation
accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
ax.text(0.5, -0.15, f'Accuracy: {accuracy:.2%}', 
        transform=ax.transAxes, ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'confusion_matrix.png', 
            dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(fpr, tpr, color='darkblue', linewidth=2, 
        label=f'ROC curve (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
ax.fill_between(fpr, tpr, alpha=0.3)

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve - Anomaly Detection', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'roc_curve.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ ROC AUC Score: {roc_auc:.3f}")

In [ ]:
# Precision-Recall Curve
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_scores)
pr_auc = auc(recall, precision)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(recall, precision, color='darkred', linewidth=2,
        label=f'PR curve (AUC = {pr_auc:.3f})')
ax.fill_between(recall, precision, alpha=0.3)

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
ax.legend(loc='lower left', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'pr_curve.png', 
            dpi=300, bbox_inches='tight')
plt.show()

### 5. Sentiment Analysis Evaluation

In [ ]:
print("\n" + "="*60)
print("SENTIMENT ANALYSIS EVALUATION")
print("="*60)

# Test cases with ground truth
sentiment_test_cases = [
    ("Excellent economic growth and tourism boom!", "positive"),
    ("Crisis deepens as fuel shortage continues", "negative"),
    ("Government announces new infrastructure project", "neutral"),
    ("Record tea exports bring prosperity to farmers", "positive"),
    ("Unemployment rate rises sharply this quarter", "negative"),
    ("Weather conditions normal across the country", "neutral"),
    ("Major breakthrough in renewable energy sector", "positive"),
    ("Power cuts cause disruption to businesses", "negative"),
]

In [ ]:
# Predict
true_labels = []
pred_labels = []
confidences = []

for text, true_label in sentiment_test_cases:
    scores = sentiment_analyzer.analyze_text(text)
    pred_label = sentiment_analyzer.get_sentiment_label(scores)
    confidence = max(scores['positive'], scores['neutral'], scores['negative'])
    
    true_labels.append(true_label)
    pred_labels.append(pred_label)
    confidences.append(confidence)

In [ ]:
# Confusion matrix for sentiment
sentiment_cm = confusion_matrix(true_labels, pred_labels, 
                                labels=['positive', 'neutral', 'negative'])

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(sentiment_cm, annot=True, fmt='d', cmap='RdYlGn',
            xticklabels=['Positive', 'Neutral', 'Negative'],
            yticklabels=['Positive', 'Neutral', 'Negative'])
ax.set_title('Confusion Matrix - Sentiment Analysis', fontsize=14, fontweight='bold')
ax.set_ylabel('True Sentiment')
ax.set_xlabel('Predicted Sentiment')

plt.tight_layout()
plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'sentiment_confusion.png', 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate accuracy
accuracy = np.trace(sentiment_cm) / sentiment_cm.sum()
print(f"\nSentiment Analysis Accuracy: {accuracy:.2%}")
print(f"Average Confidence: {np.mean(confidences):.2%}")

### 6. Risk Scoring Evaluation

In [ ]:
print("\n" + "="*60)
print("RISK SCORING EVALUATION")
print("="*60)

# Generate risk scenarios
risk_scenarios = []
for i in range(100):
    # Create varying risk levels
    if i < 20:  # High risk
        sentiment = np.random.uniform(-1, -0.5)
        volatility = np.random.uniform(0.5, 0.8)
        anomaly = np.random.uniform(0.7, 1.0)
        expected_risk = 'high'
    elif i < 50:  # Medium risk
        sentiment = np.random.uniform(-0.5, 0)
        volatility = np.random.uniform(0.3, 0.6)
        anomaly = np.random.uniform(0.4, 0.7)
        expected_risk = 'medium'
    else:  # Low risk
        sentiment = np.random.uniform(0, 1)
        volatility = np.random.uniform(0.1, 0.4)
        anomaly = np.random.uniform(0, 0.4)
        expected_risk = 'low'
    
    scenario = {
        'sentiment_score': sentiment,
        'sentiment_volatility': volatility,
        'time_series_values': np.random.randn(50),
        'trend_slope': np.random.uniform(-0.5, 0.5),
        'trend_strength': np.random.uniform(0.3, 0.9),
        'anomaly_score': anomaly,
        'source_credibility': np.random.uniform(0.7, 1.0),
        'timestamp': pd.Timestamp.now(),
        'expected_risk': expected_risk
    }
    risk_scenarios.append(scenario)

In [ ]:
# Assess all scenarios
risk_scores = []
risk_levels = []
expected_levels = []

for scenario in risk_scenarios:
    expected = scenario.pop('expected_risk')
    assessment = risk_scorer.assess_risk(scenario)
    
    risk_scores.append(assessment.overall_score)
    risk_levels.append(assessment.risk_level.value)
    expected_levels.append(expected)

In [ ]:
# Visualize risk distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Risk score histogram
axes[0].hist(risk_scores, bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[0].axvline(0.7, color='red', linestyle='--', label='High Risk Threshold', linewidth=2)
axes[0].axvline(0.4, color='orange', linestyle='--', label='Medium Risk Threshold', linewidth=2)
axes[0].set_title('Risk Score Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Risk Score')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Risk level counts
level_counts = pd.Series(risk_levels).value_counts()
colors_map = {'critical': '#EF4444', 'high': '#F59E0B', 'medium': '#F59E0B', 'low': '#10B981', 'minimal': '#10B981'}
bar_colors = [colors_map.get(level, 'gray') for level in level_counts.index]

axes[1].bar(level_counts.index, level_counts.values, color=bar_colors, edgecolor='black', linewidth=2)
axes[1].set_title('Risk Level Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'risk_distribution.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Risk Score Statistics:")
print(f"  Mean: {np.mean(risk_scores):.2f}")
print(f"  Std: {np.std(risk_scores):.2f}")
print(f"  Range: [{np.min(risk_scores):.2f}, {np.max(risk_scores):.2f}]")

### 7. Performance Metrics Summary

In [ ]:
print("\n" + "="*60)
print("OVERALL PERFORMANCE SUMMARY")
print("="*60)

# Calculate metrics
from sklearn.metrics import precision_score, recall_score, f1_score

anomaly_precision = precision_score(y_test, y_pred)
anomaly_recall = recall_score(y_test, y_pred)
anomaly_f1 = f1_score(y_test, y_pred)

metrics_summary = {
    'anomaly_detection': {
        'precision': float(anomaly_precision),
        'recall': float(anomaly_recall),
        'f1_score': float(anomaly_f1),
        'roc_auc': float(roc_auc),
        'pr_auc': float(pr_auc)
    },
    'sentiment_analysis': {
        'accuracy': float(accuracy),
        'avg_confidence': float(np.mean(confidences))
    },
    'risk_scoring': {
        'mean_score': float(np.mean(risk_scores)),
        'std_score': float(np.std(risk_scores)),
        'total_assessed': len(risk_scores)
    }
}


In [ ]:
# Print summary table
print("\nAnomaly Detection:")
print(f"  Precision: {anomaly_precision:.3f}")
print(f"  Recall:    {anomaly_recall:.3f}")
print(f"  F1-Score:  {anomaly_f1:.3f}")
print(f"  ROC AUC:   {roc_auc:.3f}")

print("\nSentiment Analysis:")
print(f"  Accuracy:       {accuracy:.2%}")
print(f"  Avg Confidence: {np.mean(confidences):.2%}")

print("\nRisk Scoring:")
print(f"  Mean Score: {np.mean(risk_scores):.2f}")
print(f"  Std Dev:    {np.std(risk_scores):.2f}")


In [ ]:
# Save metrics
import json
metrics_path = Path.cwd().parent / 'data' / 'processed' / 'evaluation_metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics_summary, f, indent=2)

print(f"\n✓ Metrics saved to: {metrics_path}")


### 8. Performance Visualization Dashboard

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('PulseX Sri Lanka - Model Performance Dashboard', 
             fontsize=18, fontweight='bold', y=0.98)

# 1. Anomaly Detection Metrics
ax1 = fig.add_subplot(gs[0, 0])
metrics = ['Precision', 'Recall', 'F1-Score']
values = [anomaly_precision, anomaly_recall, anomaly_f1]
colors = ['#3B82F6', '#10B981', '#F59E0B']
bars = ax1.bar(metrics, values, color=colors, edgecolor='black', linewidth=2)
ax1.set_ylim(0, 1)
ax1.set_title('Anomaly Detection', fontweight='bold')
ax1.set_ylabel('Score')
ax1.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 0.02, f'{val:.3f}', 
             ha='center', fontweight='bold')

# 2. ROC Curve (mini)
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(fpr, tpr, color='darkblue', linewidth=2)
ax2.plot([0, 1], [0, 1], 'k--', linewidth=1)
ax2.fill_between(fpr, tpr, alpha=0.3)
ax2.set_title(f'ROC Curve (AUC={roc_auc:.3f})', fontweight='bold')
ax2.set_xlabel('FPR')
ax2.set_ylabel('TPR')
ax2.grid(True, alpha=0.3)

# 3. Risk Score Distribution
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(risk_scores, bins=20, color='coral', edgecolor='black')
ax3.axvline(0.7, color='red', linestyle='--', linewidth=2)
ax3.axvline(0.4, color='orange', linestyle='--', linewidth=2)
ax3.set_title('Risk Score Distribution', fontweight='bold')
ax3.set_xlabel('Risk Score')
ax3.set_ylabel('Frequency')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Confusion Matrix (mini)
ax4 = fig.add_subplot(gs[1, 0])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4, cbar=False,
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'])
ax4.set_title('Confusion Matrix', fontweight='bold')

# 5. Sentiment Accuracy
ax5 = fig.add_subplot(gs[1, 1])
sentiment_data = ['Correct', 'Incorrect']
sentiment_values = [accuracy, 1-accuracy]
colors_sent = ['#10B981', '#EF4444']
pie = ax5.pie(sentiment_values, labels=sentiment_data, autopct='%1.1f%%',
              colors=colors_sent, startangle=90)
ax5.set_title(f'Sentiment Accuracy\n({accuracy:.1%})', fontweight='bold')

# 6. System Performance Overview
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')
performance_text = f"""
SYSTEM PERFORMANCE

Anomaly Detection:
  • Precision: {anomaly_precision:.1%}
  • Recall: {anomaly_recall:.1%}
  • F1-Score: {anomaly_f1:.1%}
  • ROC AUC: {roc_auc:.3f}

Sentiment Analysis:
  • Accuracy: {accuracy:.1%}
  • Confidence: {np.mean(confidences):.1%}

Risk Scoring:
  • Assessments: {len(risk_scores)}
  • Mean Score: {np.mean(risk_scores):.2f}
  • Std Dev: {np.std(risk_scores):.2f}
"""
ax6.text(0.1, 0.9, performance_text, transform=ax6.transAxes,
         fontsize=10, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# 7-9: Additional metrics
ax7 = fig.add_subplot(gs[2, :])
categories = ['Anomaly\nDetection', 'Sentiment\nAnalysis', 'Risk\nScoring', 'Overall\nSystem']
performance_scores = [
    (anomaly_precision + anomaly_recall + anomaly_f1) / 3,
    accuracy,
    1 - np.std(risk_scores),  # Lower std = better
    0.88  # Overall system performance
]

bars = ax7.barh(categories, performance_scores, 
                color=['#3B82F6', '#10B981', '#F59E0B', '#8B5CF6'],
                edgecolor='black', linewidth=2)
ax7.set_xlim(0, 1)
ax7.set_xlabel('Performance Score', fontweight='bold')
ax7.set_title('Component Performance Comparison', fontweight='bold')
ax7.grid(True, alpha=0.3, axis='x')

for bar, score in zip(bars, performance_scores):
    ax7.text(score + 0.02, bar.get_y() + bar.get_height()/2, 
             f'{score:.2%}', va='center', fontweight='bold')

plt.savefig(Path.cwd().parent / 'data' / 'processed' / 'performance_dashboard.png', 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("\n" + "="*60)
print("EVALUATION COMPLETE")
print("="*60)
print("\n✓ All visualizations saved to data/processed/")
print("✓ Metrics summary saved to evaluation_metrics.json")
print("\n🎉 Models ready for production deployment!")
print("="*60)